In [16]:
import pandas as pd
import numpy as np
import scipy

import os
import datetime

### Read in Data

In [53]:
# Read in the bid-ask data from WRDS
bid_ask_df = pd.read_sas('../Data/bid_ask_data.sas7bdat')
print(bid_ask_df.shape)
bid_ask_df.head()

(957126, 8)


,PERMNO,DATE,BID,ASK,BIDLO,ASKHI,decile_category,stock_num
0,10028.0,2000-01-03,5.6250,6.12500,5.75000,6.3750,1.0,1.0
1,10514.0,2000-01-03,10.4375,10.56250,10.12500,10.5000,1.0,3.0
2,10656.0,2000-01-03,11.0000,11.25000,11.00000,11.0000,1.0,4.0
3,10661.0,2000-01-03,1.6250,1.65625,1.37500,1.8125,1.0,5.0
4,10838.0,2000-01-03,1.0000,1.06250,0.90625,1.0625,1.0,6.0


In [110]:
# Read in the CCI data
CCI_df = pd.read_csv('../Data/CCI.csv')
print(CCI_df.shape)
CCI_df.head()

(13203, 8)


,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,NLD,CCI,AMPLITUD,LTRENDIDX,M,1991-01,99.75915,NaN
1,NLD,CCI,AMPLITUD,LTRENDIDX,M,1991-02,99.74998,NaN
2,NLD,CCI,AMPLITUD,LTRENDIDX,M,1991-03,99.88835,NaN
3,NLD,CCI,AMPLITUD,LTRENDIDX,M,1991-04,99.95676,NaN
4,NLD,CCI,AMPLITUD,LTRENDIDX,M,1991-05,100.01380,NaN


In [111]:
# Read in the ICS data
ICS_df = pd.read_csv('../Data/ICS.csv')
print(ICS_df.shape)
ICS_df.head()

(607, 3)


,Month,YYYY,ICS_ALL
0,November,1952,86.2
1,February,1953,90.7
2,August,1953,80.8
3,November,1953,80.7
4,February,1954,82.0


### Helper Functions

In [112]:
def bid_ask_estimate():
    # Question for Prof. Kramer: to compute beta (E[(ln(H_t/L_t)^2 + ln(H_t+1/L_t+1)^2)]) do we just take the mean?
    
    # beta = E{sum(j=0,1)[ln(H_(t+j)/L_(t+j))]^2} where H_(t) is the observed high price on day t (similarly for L)
    beta = ...
    
    # gamma = [ln(H_(t, t+1)/L_(t, t+1))]^2 where H_(t, t+1) is the high price over days t and t+1 (similarly for L)
    gamma = (np.log(high/low))**2
    
    # alpha = ((2beta)^0.5-beta^0.5)/(3-2*2^0.5) - (gamma/(3-2*2^0.5))^0.5
    alpha = (np.sqrt(2*beta)-np.sqrt(beta))/(3-2*np.sqrt(2))-np.sqrt(gamma/(3-2*np.sqrt(2)))
    
    # high-low spread estimate S = 2(e^alpha - 1)/(1+e^alpha)
    S = 2*(np.exp(alpha)-1)/(1+np.exp(alpha))
    
    return S

In [117]:
def split_date_field(df, date_field):
    '''
    Extracts year and month columns from date column
    
    Return: df with month and year columns
    '''
    new_df = df.copy()
    
    split_date = new_df[date_field].str.split('-', n=1, expand=True)
    
    new_df['Year'] = split_date[0].astype(int)
    new_df['Month'] = split_date[1].astype(int)
    
    return new_df
    
def get_integer_month(df, month_field, short_names=False):
    '''
    Get months as integers from string month field
    
    Return: df with updated month field to have integer values
    '''
    
    new_df = df.copy()
    
    # Check for integer months
    if pd.api.types.is_integer_dtype(df[month_field]):
        print("Month field already in integer form.")
        return df
    
    # If not integer months, convert months to integers
    if short_names:
        new_df[month_field] = new_df[month_field].apply(lambda x: datetime.datetime.strptime(x, "%b").month)
    
    else:
        new_df[month_field] = new_df[month_field].apply(lambda x: datetime.datetime.strptime(x, "%B").month)

    return new_df

In [118]:
def get_daily_data(df, trading_days, year_field, month_field, cols_to_keep=None):
    '''
    Interpolates the data on a daily basis from a monthly basis (same value between dates)
    
    Return: df with daily data
    '''
    
    trading_months = trading_days.month
    trading_years = trading_days.year
    date_df = pd.DataFrame({'Date': trading_days, 'Year': trading_years, 'Month': trading_months})
    
    if cols_to_keep:
        merged_df = date_df.merge(df[cols_to_keep+[year_field]+[month_field]], left_on=['Year', 'Month'], 
                                  right_on=[year_field, month_field], how='left')
    
    else:
        merged_df = date_df.merge(df, left_on=['Year', 'Month'], 
                                  right_on=[year_field, month_field], how='left')        
    
    return merged_df

### Pre-process the Data

In [133]:
# Extract month and year, then average the CCI data across each state
CCI_data_cleaned = split_date_field(CCI_df, 'TIME')
CCI_data_cleaned = CCI_data_cleaned.groupby(['TIME']).mean()[['Value', 'Year', 'Month']].reset_index()

# Get the integer monthly values for ICS data
ICS_data_cleaned = get_integer_month(ICS_df, 'Month')

# Get trading days
trading_days = pd.to_datetime(bid_ask_df.DATE.unique().astype(str))

# Get daily data for CCI and ICS
CCI_daily = get_daily_data(CCI_data_cleaned, trading_days, 'Year', 'Month')
ICS_daily = get_daily_data(ICS_data_cleaned, trading_days, 'YYYY', 'Month')

# Get rid of unnecessary columns, rename columns
CCI_daily = CCI_daily.drop(['Year', 'Month', 'TIME'], axis=1)
CCI_daily = CCI_daily.rename({'Value': 'CCI'}, axis='columns')

ICS_daily = ICS_daily.drop(['Year', 'Month', 'YYYY'], axis=1)
ICS_daily = ICS_daily.rename({'ICS_ALL': 'ICS'}, axis='columns')

In [141]:
# Convert date field to datetime
bid_ask_df['DATE'] = pd.to_datetime(bid_ask_df.DATE.astype(str))

# Get the absolute and relative bid-ask spread data
bid_ask_df['ABS_SPREAD'] = bid_ask_df.ASK - bid_ask_df.BID
bid_ask_df['REL_SPREAD'] = (bid_ask_df.ASK - bid_ask_df.BID)*2/(bid_ask_df.ASK + bid_ask_df.BID)

# Get the equal-weighted averages of firm-specific daily closing quoted spreads
bid_ask_daily = bid_ask_df.groupby(['DATE', 'decile_category'])

In [142]:
bid_ask_df.head()

,PERMNO,DATE,BID,ASK,BIDLO,ASKHI,decile_category,stock_num,ABS_SPREAD,REL_SPREAD
0,10028.0,2000-01-03,5.6250,6.12500,5.75000,6.3750,1.0,1.0,0.50000,0.085106
1,10514.0,2000-01-03,10.4375,10.56250,10.12500,10.5000,1.0,3.0,0.12500,0.011905
2,10656.0,2000-01-03,11.0000,11.25000,11.00000,11.0000,1.0,4.0,0.25000,0.022472
3,10661.0,2000-01-03,1.6250,1.65625,1.37500,1.8125,1.0,5.0,0.03125,0.019048
4,10838.0,2000-01-03,1.0000,1.06250,0.90625,1.0625,1.0,6.0,0.06250,0.060606
